* Compile, Train and Save the models here

* 1. Preprocessing

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # CSV file
import config
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import davies_bouldin_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

In [2]:
def Scale_and_PCALDA(path):

    data = pd.read_csv(path)
    num_columns = data.shape[1]
    print(f"Num of Columns is {num_columns}")
    X = np.array(data.iloc[:,0:num_columns-1])
    y = np.array(data.iloc[:,num_columns-1])
    # print(len(X[0]))
    # print(y[0])
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  # X shape: (n_samples, 12)

    ''' PCA '''
    n_components = 2
    pca_object = PCA(n_components= n_components)
    pca_object.fit(X_scaled)
    PrincipleComps = pca_object.transform(X_scaled)
    classes = np.unique(y)

    for i in range(n_components):
        plt.figure()
        for clss in classes:
            plt.hist(PrincipleComps[y == clss, i],
                    bins="auto", alpha=0.5, 
                    label=f"Class {clss}")
        plt.xlabel("Feature intervals")
        plt.ylabel("Frequency")
        plt.title(f"PCA by Class for feature column {i}")
        plt.legend()
        plt.grid(True)
        plt.show()
    score = davies_bouldin_score(PrincipleComps, y)
    print(f"The davies_bouldin_score for PCA is {score}")


    """ LDA """

    lda_mcc = LDA()
    lda_mcc.fit(X_scaled,y)
    lda_OP = lda_mcc.transform(X_scaled)
    plt.figure()
    for c in classes:
        plt.hist(lda_OP[y == c], bins=20, alpha=0.5, label=f"Class {c}")
    plt.xlabel("1D LDA Projection")
    plt.ylabel("Frequency")
    plt.title("LDA Projection onto First Component. 0 is cat, 1 is Dog")
    plt.legend()
    plt.grid(True)
    plt.show()

In [3]:

# print("Original DATA")
# Scale_and_PCALDA(path = config.Features + 'data.csv' )

# print("fs300_cc12")
# Scale_and_PCALDA(path = config.Features + 'data_fs300_cc12.csv')

# print("fs300_cc20")
# Scale_and_PCALDA(path = config.Features + 'data_fs300_cc20.csv')

# print("fs300_cc30")
# Scale_and_PCALDA(path = config.Features + 'data_fs300_cc30.csv')

# print("fs500_cc20")
# Scale_and_PCALDA(path = config.Features + 'data_fs500_cc20.csv')

In [11]:
def Load_and_Train(data, Epochs): 
    num_columns = data.shape[1]
    X = data.iloc[:,0:num_columns-1]
    y = data.iloc[:,num_columns-1]
    mask = X.iloc[:, -4:].sum(axis=1) != 0
    X = np.array(X[mask])
    y = np.array(y[mask])

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y,
        test_size=0.2,         
        stratify=y    
    )

    model = Sequential([

        Dense(96, input_shape=(20,), activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation='tanh'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile the model
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

    # Early stopping to prevent overfitting
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train,
                        validation_split=0.2,
                        epochs=Epochs,
                        batch_size=32,
                        callbacks=[early_stop],
                        verbose=1)
    print('')
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy:.2f}")

* 2. Training the model

* First approach- Vanilla NN 
* fs300_cc20 looks good. Let us see....................

In [12]:
data = pd.read_csv(config.Features + 'data_fs300_cc20.csv')
Load_and_Train(data, 80)


Epoch 1/80


c:\Users\desly\ml_lab\lab-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


185/185 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6778 - loss: 0.6699 - val_accuracy: 0.8358 - val_loss: 0.3885
Epoch 2/80
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8058 - loss: 0.4516 - val_accuracy: 0.8501 - val_loss: 0.3435
Epoch 3/80
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8185 - loss: 0.4302 - val_accuracy: 0.8596 - val_loss: 0.3231
Epoch 4/80
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8374 - loss: 0.3994 - val_accuracy: 0.8711 - val_loss: 0.3066
Epoch 5/80
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8425 - loss: 0.3888 - val_accuracy: 0.8738 - val_loss: 0.2939
Epoch 6/80
185/185 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8457 - loss: 0.3671 - val_accuracy: 0.8813 - val_loss: 0.2857
Epoch 7/80
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8440 - loss: 0.3638 - val_accuracy: 0.8881 - val_loss: 0.2756
Epoch 8/80
185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8500 - loss: 0.3571 - val_accuracy: 0.8955 - val_

* Second approach- Vanilla NN and fs500_cc20 dataset

In [13]:
data = pd.read_csv(config.Features + 'data_fs500_cc20.csv')
Load_and_Train(data, 80)

Epoch 1/80


c:\Users\desly\ml_lab\lab-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


179/179 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6759 - loss: 0.6285 - val_accuracy: 0.8308 - val_loss: 0.4282
Epoch 2/80
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8107 - loss: 0.4426 - val_accuracy: 0.8538 - val_loss: 0.3655
Epoch 3/80
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8297 - loss: 0.4056 - val_accuracy: 0.8622 - val_loss: 0.3458
Epoch 4/80
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8402 - loss: 0.4003 - val_accuracy: 0.8720 - val_loss: 0.3273
Epoch 5/80
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8478 - loss: 0.3776 - val_accuracy: 0.8790 - val_loss: 0.3130
Epoch 6/80
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8524 - loss: 0.3646 - val_accuracy: 0.8797 - val_loss: 0.3048
Epoch 7/80
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8584 - loss: 0.3416 - val_accuracy: 0.8783 - val_loss: 0.2904
Epoch 8/80
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8627 - loss: 0.3561 - val_accuracy: 0.8846 - val_

* 3rd approach- Vanilla NN and Random sampled dataset fs300_cc20

In [14]:
data = pd.read_csv(config.Features + 'Rdata_fs300_cc20.csv')
Load_and_Train(data, 80)

Epoch 1/80


c:\Users\desly\ml_lab\lab-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


654/654 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7689 - loss: 0.5188 - val_accuracy: 0.8902 - val_loss: 0.2812
Epoch 2/80
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8697 - loss: 0.3335 - val_accuracy: 0.9047 - val_loss: 0.2540
Epoch 3/80
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8812 - loss: 0.3109 - val_accuracy: 0.9103 - val_loss: 0.2387
Epoch 4/80
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8894 - loss: 0.2891 - val_accuracy: 0.9131 - val_loss: 0.2299
Epoch 5/80
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8909 - loss: 0.2850 - val_accuracy: 0.9195 - val_loss: 0.2166
Epoch 6/80
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8924 - loss: 0.2758 - val_accuracy: 0.9196 - val_loss: 0.2086
Epoch 7/80
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8960 - loss: 0.2680 - val_accuracy: 0.9233 - val_loss: 0.2072
Epoch 8/80
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8943 - loss: 0.2635 - val_accuracy: 0.9233 - val_

* Approach 3 - 2D convolutional model and fs300cc20f20 dataset

In [2]:
def min_max_normalize(feature):
    min_val = np.min(feature)
    max_val = np.max(feature)
    return (feature - min_val) / (max_val - min_val + 1e-8)  # add epsilon to avoid division by zero


In [3]:
features = np.load(config.Features + '2Ddata_fs300_cc20_f50/features.npy')  
labels = np.load(config.Features + '2Ddata_fs300_cc20_f50/labels.npy')    
# print(features[0])
X_scaled = min_max_normalize(features)
print(X_scaled[0].shape)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2)

(50, 20)


In [4]:

# Define input shape
input_shape = (features.shape[1], features.shape[2], 1)  # Add channel dimension for CNNs

# Reshape features for CNN: (samples, height, width, channels)
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# Example model structure
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # for binary classification (cat vs dog)
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.summary()

c:\Users\desly\ml_lab\lab-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 49, 19, 32)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 9, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6912)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       442,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 442,657 (1.69 MB)

 Trainable params: 442,657 (1.69 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train
model.fit(X_train, y_train, 
          epochs=200, 
          validation_data=(X_test, y_test),
          batch_size=32,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

Epoch 1/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5189 - loss: 0.7251 - val_accuracy: 0.6029 - val_loss: 0.6713
Epoch 2/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5866 - loss: 0.6772 - val_accuracy: 0.6029 - val_loss: 0.6656
Epoch 3/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6213 - loss: 0.6577 - val_accuracy: 0.6029 - val_loss: 0.6679
Epoch 4/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5762 - loss: 0.6679 - val_accuracy: 0.6029 - val_loss: 0.6463
Epoch 5/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5855 - loss: 0.6517 - val_accuracy: 0.6029 - val_loss: 0.6206
Epoch 6/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5683 - loss: 0.6253 - val_accuracy: 0.6029 - val_loss: 0.5739
Epoch 7/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6172 - loss: 0.5718 - val_accuracy: 0.7653 - val_loss: 0.5256
Epoch 8/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6990 - loss: 0.5154 - val_accuracy: 0.7581

In [6]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9795 - loss: 0.0505 
Test Accuracy: 0.99
